In [37]:
import pandas as pd
import numpy as np
import pprint as pprint
import time


### Loadind data from csv files into datafreames


In [38]:
# Restaurants
dfaccepts=pd.read_csv(filepath_or_buffer='Restaurant-data/chefmozaccepts.csv',encoding='ascii')
dfcuisine=pd.read_csv(filepath_or_buffer='Restaurant-data/chefmozcuisine.csv',encoding='utf8')
dfhours4=pd.read_csv(filepath_or_buffer='Restaurant-data/chefmozhours4.csv',encoding='utf8')
dfparkingdata2=pd.read_csv('Restaurant-data/chefmozparking.csv')
dfgeoplaces=pd.read_csv(filepath_or_buffer='Restaurant-data/geoplaces2.csv',encoding='latin-1')
# # consumers
dfusercuisine=pd.read_csv('Restaurant-data/usercuisine.csv')
dfuserpayment=pd.read_csv('Restaurant-data/userpayment.csv')
dfuserprofile=pd.read_csv('Restaurant-data/userprofile.csv')
# user item rating
dfrating=pd.read_csv('Restaurant-data/rating_final.csv')


###  Data cleaning

In [42]:
# To perform general cleaning on all datasets and inspecting
def cleandf(data):
    data.replace('?',np.NaN,inplace=True)
#     print('no. of DUBLICATES', data.duplicated().sum(axis=0))
#     print('NULL ROWS',data.isnull().sum(axis=0))
    data.drop_duplicates(inplace = True)
    adult['sex'].map(lambda x: x.lower()).head()
#     print(data.shape)
    return 0
cleandf(dfaccepts)
cleandf(dfgeoplaces)
cleandf(dfparkingdata2)
cleandf(dfhours4)
cleandf(dfcuisine)
cleandf(dfusercuisine)
cleandf(dfuserpayment)
cleandf(dfuserprofile)
cleandf(dfrating)




0

In [43]:
# data cleaning geoplaces
def cleandfgeoplaces():
    dfgeoplaces.isnull().sum(axis=0)
    # Drop null collunms bellow a given threshhod here 74. Removes zip,url and fax
    dfgeoplaces.dropna(thresh=74,axis=1,inplace=True)
    #removing spaces
    stripped=dfgeoplaces.select_dtypes(include='object').apply(lambda x: x.str.strip())
#resolve city naming diferencesas a result of typo 
    typofreeCity= stripped.city.map(lambda x : 
            'San Luis Potosi' if (x=='s.l.p.') |(x=='s.l.p')| (x=='san luis potos' )| (x=='slp') | (x=='san luis potosi')else
            'Ciudad Victoria' if (x=='Cd. Victoria') | (x=='victoria') | (x=='Cd Victoria') | (x=='Victoria') else x)
    stripped.city=typofreeCity
#resolve state naming diferencesas a result of typo 
    typofreeState= stripped.state.map(lambda x : 
            'San Luis Potosi' if (x=='s.l.p.') |(x=='s.l.p')| (x=='san luis potos' )| (x=='slp') | (x=='san luis potosi')| (x=='SLP') | (x=='S.L.P.')else
            'Ciudad Victoria' if (x=='Cd. Victoria') | (x=='victoria') | (x=='Cd Victoria') | (x=='Victoria') else x)
    stripped.state=typofreeState
# fill all null country collumn with mexico
    stripped.country.fillna('Mexico')
    dfgeoplaces[list(dfgeoplaces.select_dtypes(include='object').columns.values)]=stripped
    return 0
cleandfgeoplaces()



0

In [44]:
import re
def dfhours4clean():
#    removing multiple ranges data fro hours collumn
    dfhours4.hours=dfhours4.hours.map(lambda x:re.split(';',x)[0])
#     dfhours4.hours.
    fromto = pd.DataFrame(dfhours4.hours.str.split('-',1).tolist(), columns = ['fromhour','tohour'])
    hourRange=pd.concat([dfhours4, fromto], axis=1, sort=False)
    return hourRange 
dfhours4clean()
dfhours41=dfhours4clean()


### Queries

#### Assignment2  Part 1:

In [45]:
# 2.1.3. What are the names of different restaurants in the state of ‘tamaulipas’?
# select where name is tamaulipas
dfgeoplaces.loc[:,['name']][(dfgeoplaces.state=='tamaulipas')]


,name
3,little pizza Emilio Portes Gil
19,TACOS EL GUERO
73,palomo tec
76,TACOS CORRECAMINOS
103,Carnitas Mata Calle 16 de Septiembre
106,puesto de gorditas
109,carnitas mata calle Emilio Portes Gil
122,tacos abi
123,la perica hamburguesa


In [46]:
# 2.1.4. How many different customers used public transport for going to therestaurants?
len(dfuserprofile[dfuserprofile['transport']=='public'])


82

In [47]:
# 2.1.5. What is the least popular payment method among customers?
min(dfuserpayment.groupby('Upayment'))


('American_Express',     userID          Upayment
 35   U1031  American_Express
 48   U1041  American_Express
 119  U1093  American_Express)

In [48]:
# 2.1.6. How many (different) restaurants work until 19:00 in the evenings?

# Clean the hours dataframe before using it to query
dfcleanHours4=dfhours4clean()
len(dfhours41[['placeID']][dfhours41.tohour=='19:00'])


48

In [49]:
# 2.1.7. Which type of food (rcuisine) is served most among the restaurants?
dfcuisine[['Rcuisine']].mode()


,Rcuisine
0,Mexican


In [50]:
# 2.1.8. What is the percentage of customers who were born between 1980 and 1990?
len(dfuserprofile[(dfuserprofile.birth_year>=1980) & (dfuserprofile.birth_year<=1990)])/len(dfuserprofile)*100

63.04347826086957

In [51]:
# 2.1.9. What is the percentage of students with a medium budget preferring walking to the restaurants?
len(dfuserprofile[(dfuserprofile.budget=='medium') &(dfuserprofile.transport=='on foot')])/len(dfuserprofile[(dfuserprofile.budget=='medium')])*100


9.89010989010989

#### Assignment2  Part 2:

In [52]:
# 2.2.1. What are the names of restaurants that do not have public parking lots?
dfgeoplaces[['placeID','name']].merge(dfparkingdata2[dfparkingdata2.parking_lot!='public'],how='inner',on='placeID').name
# dfgeoplaces

0                                        Kiku Cuernavaca
1                                        puesto de tacos
2                             El Rincón de San Francisco
3                         little pizza Emilio Portes Gil
4                                          carnitas_mata
5                               Restaurant los Compadres
6                                      Taqueria EL amigo
7                                              shi ro ie
8                               Pollo_Frito_Buenos_Aires
9                                   la Estrella de Dimas
10                                        Restaurante 75
11                             Abondance Restaurante Bar
12                                  El angel Restaurante
13                             Restaurante Pueblo Bonito
14                           Mcdonalds Parque Tangamanga
15                        Tortas y hamburguesas el gordo
16                                               Sirlone
17                             

In [53]:
# 2.2.2. What are the addresses of restaurants which only accept ‘cash’?
dfgeoplaces.merge(dfaccepts[dfaccepts.Rpayment=='cash'],on='placeID',how='inner').address


0         esquina santos degollado y leon guzman
1                                Universidad 169
2                        calle emilio portes gil
3                         lic. Emilio portes gil
4                 Camino a Simon Diaz 155 Centro
5               Calle Mezquite Fracc Framboyanes
6                                            NaN
7                                        tampico
8              Villa de Pozos 192 Villa de Pozos
9             Villa de Pozos 4497 Villa de Pozos
10                   Industrias 908 Valle Dorado
11               Venustiano Carranza 1625 Jardin
12                       Mexico 2015 Providencia
13           Lateral Salvador Nava Martinez 3145
14                              Ricardo B. Anaya
15                                  carr. mexico
16                           agustin de iturbide
17                              Plaza del Carmen
18                De Guadalupe 460 San Miguelito
19                               Av. V. Carranza
20                  

In [54]:
# 2.2.3. Name the cities where the restaurants cook and serve ‘fast food’?

dfgeoplaces[['placeID','city']].merge(dfcuisine[dfcuisine.Rcuisine=='Fast_Food'],how='inner',on='placeID').city ##names not distict=== to add distinct


0    Ciudad Victoria
1    San Luis Potosi
2    San Luis Potosi
3    San Luis Potosi
4         Cuernavaca
5                NaN
6            Soledad
7    San Luis Potosi
Name: city, dtype: object

In [55]:
# 2.2.4. What is the most common ‘rating’ among customers with family?

dfrating[['userID','rating']].merge(dfuserprofile[dfuserprofile.ambience=='family'],on='userID',how='inner').rating.mode()


0    2
dtype: int64

In [56]:
# 2.2.5. What types of food (‘rcuisine’) received the highest rank from people with‘low’ budget?

#filter each dataset by conditions and merging the filtered datasets 
food=dfcuisine.merge(dfrating[['userID','placeID','food_rating']],on='placeID',how='left').merge(dfuserprofile[['userID','budget']][dfuserprofile.budget=='low'],on='userID',how='inner')
#grouping the merged output by rquisin and getting sum for each cuisine
groupedSum=food.groupby('Rcuisine').sum()
#get the maximum Id value based on the collum foodrating
groupedSum.food_rating.idxmax()


'Mexican'

In [57]:
# 2.2.6. What is the average of ‘service rating’, received from ‘social drinkers’ about restaurants which just served ‘Wine-Beer’?

# getting service rating for social drinkers
label=dfrating[['userID','placeID','service_rating']].merge(dfuserprofile[['drink_level','userID']][dfuserprofile.drink_level=='social drinker'],on='userID',how='inner')
label2=label.merge(dfgeoplaces[['placeID']][dfgeoplaces.alcohol=='Wine-Beer'],on='placeID',how='inner')
#calculating the average
label2.service_rating.mean(axis=0)


1.3863636363636365

In [59]:
# 2.2.7. How many smokers gave zero ‘service rating’ to the restaurants without an open area?
len(dfuserprofile[dfuserprofile.smoker=='true'].merge(dfrating[dfrating.service_rating==0], on='userID',how='inner').merge(dfgeoplaces[dfgeoplaces.area!='open'],on='placeID',how='inner'))


44

In [60]:
# 2.2.8. Find the correlation between different rating categories ((general) rating, food_rating, service_rating) with the price levels of the restaurants.
# Hint: To answer the last question, you need to change the type of attribute values

restaurantsRating=dfgeoplaces[['placeID','price']].merge(dfrating, on='placeID', how='inner')
# from categorical to numerical.
price=restaurantsRating.price.map(lambda x: 1 if x == 'low' else 2 if x=='medium' else 3 ) # let 1=low ,2=medium,3=high

print('between price and general rating= ',price.corr(restaurantsRating.rating))
print('between price and food rating= ',price.corr(restaurantsRating.food_rating))
print ('between price and Service rating= ',price.corr(restaurantsRating.service_rating))


between price and general rating=  0.09074519223574697
between price and food rating=  0.0510812284528916
between price and Service rating=  0.11869614600979307
